In [39]:
import tkinter as tk            # Imports tkinter for the GUI itself
from tkinter import messagebox  # Imports the tkinter messagebox function from tkinter for making message boxes
import random                   # Imports random for the opponent AI

# This is the Main Window, primarily the title of it and the window's color
root = tk.Tk()
root.title("Tic-Tac-Toe")
root.configure(bg="gray10") # this sets the background color to a dark gray because dark mode aesthetic is preferred

# The board setup where player is determined as "X", sets the initial board spaces and initializes the x and o score values.
player = "X"
starting_player = "Human"  # This tracks the starting player, by default its the human player
board_size = 3
board = [""] * (board_size * board_size)
x = 0
o = 0

# Button or panel click handler for clicking the panels and allow for them to be marked with "X" or "O"
def panel_click(index):
    global player
    if board[index] == "":  # checks if panel is empty
        board[index] = player
        panels[index].config(text=player)
        # this is the win/lose condition that is triggered by the player
        if winner():
            score_updater()
            messagebox.showinfo("Game Over", f"{player} wins!")
            restart_board()
        elif "" not in board:
            messagebox.showinfo("Game Over", "Uh oh, looks like its a tie!")
            restart_board()
        else:
            player = "O" if player == "X" else "X"
            if player == "O":
                opponent()

# Sets up the Oponent's conditions
def opponent():
    global player
    move = opponent_logic()
    if move is not None:
        board[move] = player
        panels[move].config(text=player)
        # this is win/lose condition that is triggered by the opponent
        if winner():
            score_updater()
            messagebox.showinfo("Game Over", f"{player} wins!")
            restart_board()
        elif "" not in board:
            messagebox.showinfo("Game Over", "Uh oh, looks like its a tie!")
            restart_board()
        else:
            player = "X"

# This is the logic that Opponent uses to compete with the player
def opponent_logic():
    available_moves = [i for i, cell in enumerate(board) if cell == ""] #grabs the list of available moves
    for move in available_moves:
        board[move] = "O"
        if winner():    # Checks if the move gives a win
            board[move] = ""
            return move # Returns with a move that gives the win
        board[move] = ""
    for move in available_moves:   
        board[move] = "X"
        if winner():    # checks if the player's winning move can be blocked
            board[move] = ""
            return move # returns with a move that blocks the win
        board[move] = ""
    return random.choice(available_moves) if available_moves else None  # randomizes movement if there's no winning move to utilize or block from the player

# This defines the function for the winning criteria
def winner():
    winning_patterns = get_winning_patterns()   # this calls the get_winning_patterns definition
    for pattern in winning_patterns:
        if all(board[i] == player for i in pattern):
            return True     # if winning pattern exists
    return False            # if winning pattern doesn't exist

# This has the winning patterns which ranges from a filled row, column, or diagonal, will work with any size board.
def get_winning_patterns():
    patterns = []
    for i in range(board_size):  
        patterns.append([i * board_size + j for j in range(board_size)])  # Rows
        patterns.append([j * board_size + i for j in range(board_size)])  # Columns
        patterns.append([i * board_size + i for i in range(board_size)])  # Diagonal Left \
        patterns.append([(i + 1) * (board_size - 1) for i in range(board_size)])  # Diagonal Right /
    return patterns

# This defines the function to restart the board whenever someone wins or there's a tie
def restart_board():
    global board, player
    board = [""] * (board_size * board_size)    # This empties the board
    player = "O" if starting_player == "Opponent" else "X" 
    for panel in panels:
        panel.config(text="")
    if player == "O":
        opponent()


# Sets the starting player, which can be changed in the program's settings menu
def set_starting_player(player_choice):
    global starting_player
    starting_player = player_choice
    settings_checkmark()
    restart_board()

# Board size modifier for switching its board size
def board_size_modifier(size):
    global board_size, panels, score_label
    board_size = size
    settings_checkmark()
    for panel in panels:
        panel.destroy()
    panels = []
    for i in range(board_size * board_size):
        panel = tk.Button(root, text="", font=("Comic Sans MS", 20), width=5, height=2, fg="white", bg="gray14", command=lambda i=i: panel_click(i))
        panel.grid(row=i // board_size, column=i % board_size, padx=5, pady=5)
        panels.append(panel)
    score_label.grid_forget()  # Clears score label
    score_label.grid(row=board_size, column=0, columnspan=board_size, pady=10)
    restart_board()

# This adds a checkmark for the selected items to the settings menus to show what is applied
def settings_checkmark():
    # Checkmarks for the selected board sizes
    size_menu.entryconfig(0, label=f"3x3 {'✓' if board_size == 3 else ''}")
    size_menu.entryconfig(1, label=f"4x4 {'✓' if board_size == 4 else ''}")
    size_menu.entryconfig(2, label=f"5x5 {'✓' if board_size == 5 else ''}")
    #Checkmarks for the selected starting player
    start_menu.entryconfig(0, label=f"Human {'✓' if starting_player == 'Human' else ''}")
    start_menu.entryconfig(1, label=f"Opponent {'✓' if starting_player == 'Opponent' else ''}")

# Menu for changing settings, and it should also reset the board
menu = tk.Menu(root)
root.config(menu=menu)
settings_menu = tk.Menu(menu, tearoff=0)
menu.add_cascade(label="Settings", menu=settings_menu)

# This is the Menu selection for the board size
size_menu = tk.Menu(settings_menu, tearoff=0)
settings_menu.add_cascade(label="Change Board Size", menu=size_menu)
size_menu.add_command(label="3x3", command=lambda: board_size_modifier(3))
size_menu.add_command(label="4x4", command=lambda: board_size_modifier(4))
size_menu.add_command(label="5x5", command=lambda: board_size_modifier(5))

# This is the Menu selection for the starting player
start_menu = tk.Menu(settings_menu, tearoff=0)
settings_menu.add_cascade(label="Starting Player", menu=start_menu)
start_menu.add_command(label="Human", command=lambda: set_starting_player("Human"))
start_menu.add_command(label="Opponent", command=lambda: set_starting_player("Opponent"))
settings_checkmark()  # This initializes the settings_checkmark definition to add the checkmarks next to the menu items



# This is the score label that tallies how many wins the player/opponent won
def score_updater():
    global x, o
    if player == "X":
        x += 1
    else:
        o += 1
    score_label.config(text=f"Human Player: {x}  Computer Player: {o}")
score_label = tk.Label(root, text=f"Human Player: {x}  Computer Player: {o}", font=("Comic Sans MS", 16), fg="white", bg="gray10")
score_label.grid(row=board_size, column=0, columnspan=board_size, pady=10)

# This creates the panels for the board for panel_click to interact with and the arrangement for it giving that layout
panels = []
for i in range(board_size * board_size):
    panel = tk.Button(root, text="", font=("Comic Sans MS", 20), width=5, height=2, fg="white", bg="gray14", command=lambda i=i: panel_click(i))
    panel.grid(row=i // board_size, column=i % board_size, padx=5, pady=5)
    panels.append(panel)

# Main event loop for the GUI to work
root.mainloop()
